In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
customer_df = pd.read_csv('olist_customers_dataset.csv')
customer_df.shape
customer_df['customer_unique_id'].nunique()

#view of original cust_df

96096

In [3]:
orders_df = pd.read_csv('olist_orders_dataset.csv')
orders_df.head(50)
orders_df.shape

#view of original orders_df

(99441, 8)

In [4]:
orderitems_df = pd.read_csv('olist_order_items_dataset.csv')
orderitems_df.shape
orderitems_df['order_id'].nunique()

#view of original order_items_df - 112650 is fine

98666

In [5]:
products_df = pd.read_csv('olist_products_dataset.csv')
products_df.head(5)
products_df['product_id'].nunique()

#og products df

32951

In [6]:
product_transl_df = pd.read_csv('product_category_name_translation.csv')
product_transl_df.head(5)
product_transl_df.shape


(71, 2)

In [7]:
cust_orders_df = pd.merge(customer_df, orders_df, on = 'customer_id')
cust_orders_df.shape

cust_orders_df['customer_unique_id'].nunique()

#view of original custumer orders df

96096

In [8]:
orders_and_items = pd.merge(orders_df, orderitems_df, on='order_id')
orders_and_items.head(5)
orders_and_items.shape

#view of orders + items in that order

(112650, 13)

In [9]:
prod_order_items = pd.merge(orders_and_items, products_df, on = 'product_id', how='right')
prod_order_items.head(5)

prod_order_items.shape

# view of orders + items in that order + what each item is

(112650, 21)

In [10]:
prod_order_items = pd.merge(prod_order_items, product_transl_df, on='product_category_name', how='left')
prod_order_items.drop(['product_category_name'], axis=1, inplace=True)
prod_order_items.head(50)

prod_order_items.shape
# same as above except adding product translation + dropping portuguese description

(112650, 21)

In [11]:
final_df = pd.merge(prod_order_items, customer_df, on = 'customer_id', how='right')
final_df.head(50)


final_df['customer_unique_id'].nunique()
#adding customer_unique_id

96096

In [12]:
cols = final_df.columns.tolist()
cols

['order_id',
 'customer_id',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date',
 'order_item_id',
 'product_id',
 'seller_id',
 'price',
 'freight_value',
 'product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm',
 'product_category_name_english',
 'customer_unique_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state']

In [13]:
cols = ['order_id',
 'customer_id',
 'customer_unique_id',
 'customer_zip_code_prefix',
 'customer_city',
 'customer_state',
 'order_status',
 'order_purchase_timestamp',
 'order_approved_at',
 'order_delivered_carrier_date',
 'order_delivered_customer_date',
 'order_estimated_delivery_date',
 'order_item_id',
 'product_id',
 'product_category_name_english',
 'seller_id',
 'price',
 'freight_value',
 'product_name_lenght',
 'product_description_lenght',
 'product_photos_qty',
 'product_weight_g',
 'product_length_cm',
 'product_height_cm',
 'product_width_cm']

In [14]:
final_df = final_df[cols]

In [15]:
final_df.head(5)
final_df.shape
final_df['customer_unique_id'].nunique()


96096

In [16]:
test = pd.DataFrame(final_df.groupby(['customer_unique_id'])['order_id'].count())
test.shape

# final_df['customer_unique_id'].nunique() -- num of unique doesnt match Kaggle but i think Kaggle wrong

(96096, 1)

In [17]:
test.columns = ['num_orders']

In [18]:
more_than_one = test.query('num_orders > 1').shape

In [19]:
'Out of {} customers in this dataset, {} customers had more than 1 order - which is {}% of all customers.'.format(test.shape[0], more_than_one[0], round((more_than_one[0]/test.shape[0])*100,2))

'Out of 96096 customers in this dataset, 11869 customers had more than 1 order - which is 12.35% of all customers.'

In [20]:
final_df.dtypes

order_id                          object
customer_id                       object
customer_unique_id                object
customer_zip_code_prefix           int64
customer_city                     object
customer_state                    object
order_status                      object
order_purchase_timestamp          object
order_approved_at                 object
order_delivered_carrier_date      object
order_delivered_customer_date     object
order_estimated_delivery_date     object
order_item_id                    float64
product_id                        object
product_category_name_english     object
seller_id                         object
price                            float64
freight_value                    float64
product_name_lenght              float64
product_description_lenght       float64
product_photos_qty               float64
product_weight_g                 float64
product_length_cm                float64
product_height_cm                float64
product_width_cm

In [21]:
final_df['order_purchase_timestamp'] = pd.to_datetime(final_df['order_purchase_timestamp'])

In [22]:
print('Data is from {} to {}.'.format(str(final_df['order_purchase_timestamp'].min()), str(final_df['order_purchase_timestamp'].max())))


Data is from 2016-09-04 21:15:19 to 2018-09-03 09:06:57.


In [23]:
#Average num of items in each order

tt = pd.DataFrame(final_df.groupby(['order_id'])['order_id'].count())
tt['order_id'].mean()


#amongst all customers, people tend to order around 1 item per order

1.1417306873695092

In [24]:
test.query('num_orders > 1').mean()

#amongst those customers that order more than 1 item, they usually only do around 2-3

num_orders    2.451681
dtype: float64

In [25]:
final_df.head()

,order_id,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,...,seller_id,price,freight_value,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,...,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,8.72,40.0,268.0,4.0,500.0,19.0,8.0,13.0
1,128e10d95713541c87cd1a2e48201934,a20e8105f23924cd00833fd87daa0831,3a51803cc0d012c3b5dc8b7528cb05f7,3366,sao paulo,SP,delivered,2017-08-15 18:29:31,2017-08-15 20:05:16,2017-08-17 15:28:33,...,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0
2,0e7e841ddf8f8f2de2bad69267ecfbcf,26c7ac168e1433912a51b924fbd34d34,ef0996a1a279c26e7ecbd737be23d235,2290,sao paulo,SP,delivered,2017-08-02 18:24:47,2017-08-02 18:43:15,2017-08-04 17:35:43,...,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,7.78,40.0,268.0,4.0,500.0,19.0,8.0,13.0
3,bfc39df4f36c3693ff3b63fcbea9e90a,53904ddbea91e1e92b2b3f1d09a7af86,e781fdcc107d13d865fc7698711cc572,88032,florianopolis,SC,delivered,2017-10-23 23:26:46,2017-10-25 02:14:11,2017-10-27 16:48:46,...,3504c0cb71d7fa48d967e0e4c94d59d9,29.99,14.10,40.0,268.0,4.0,500.0,19.0,8.0,13.0
4,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,...,289cdb325fb7e7f891c38608bf9e0962,118.70,22.76,29.0,178.0,1.0,400.0,19.0,13.0,19.0
